In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
import numpy as np
import pylab as pl
import pandas as pd
import requests
from io import BytesIO

In [8]:
key='1ud8n0zzmJgrV6re45F9gsH2I9uEmSGS_oYHaTXx9E88'

In [9]:
def extract(key):
    link = 'https://docs.google.com/spreadsheet/ccc?key=' + key + '&output=csv'

    r = requests.get(url=link)
    data = r.content

    df = pd.read_table(BytesIO(data),header=None,sep=',')

    return df

In [10]:
# df = extract(key)
df = pd.read_excel('Estrutura.xlsx', header=None)
df

,0,1,2,3,4,5,6
0,6,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,4.0,0.0,0.0,0.0,1.0,0.0
3,2,8.0,0.0,0.0,1.0,1.0,0.0
4,3,12.0,0.0,0.0,0.0,0.0,0.0
5,4,8.0,3.0,0.0,0.0,0.0,0.0
6,5,4.0,3.0,0.0,0.0,0.0,0.0
7,9,NaN,NaN,NaN,NaN,NaN,NaN
8,0,0.0,1.0,NaN,NaN,NaN,NaN
9,1,2.0,3.0,NaN,NaN,NaN,NaN


In [ ]:
def transform(data, log=False):
    nodes = {}
    supports = {}
    conec = []
    loads = {}

    pos = 0

    n_nodes = data.iloc[0, 0]
    pos = pos + 1

    for i in range(pos, pos + n_nodes):
        node_id = str(data.iloc[i, 0])
        nodes[node_id] = [data.iloc[i, 1], data.iloc[i, 2]]
        supports[node_id] = (data.iloc[i, 4], data.iloc[i, 5], data.iloc[i, 6])
        pos = pos + 1

    n_conec = data.iloc[pos, 0]
    pos = pos + 1

    for i in range(pos, pos + n_conec):
        conec.append([str(int(data.iloc[i, 1])), str(int(data.iloc[i, 2]))])
        pos = pos + 1

    n_loads = data.iloc[pos, 0]
    pos = pos + 1

    for i in range(pos, pos + n_loads):
        node_id = str(data.iloc[i, 0])
        if loads.get(node_id) is None:
            loads[node_id] = [data.iloc[i, 1], data.iloc[i, 2], data.iloc[i, 3]]
        else:
            loads[node_id][0] += data.iloc[i, 1]
            loads[node_id][1] += data.iloc[i, 2]
            loads[node_id][2] += data.iloc[i, 3]
        pos = pos + 1

    if log:
        print("Nodes: \n")
        print(nodes)
        print("\nSupports: \n")
        print(supports)
        print("\nConections: \n")
        print(conec)
        print("\nLoads: \n")
        print(loads)

    structure_data = {'nodes':nodes, 'supports':supports, 'conec':conec, 'loads':loads}
    # print(structure_data)

    return structure_data